In [ ]:
# K-IFRS PDF -> 단일 JSON (문단번호/페이지/내용 포함, 후처리 규칙 반영)
from pathlib import Path
import re, json
from typing import List, Dict, Any, Tuple

# ▼ PDF들이 있는 폴더 경로 (필요시 수정)
PDF_DIR = Path('.../raws/raws_K-ifrs')
# ▼ 출력 파일 (단일 JSON)
OUT_PATH = Path('.../raws/raws_K-ifrs/kifrs_combined_2.json')

print('PDF_DIR =', PDF_DIR.resolve())
print('OUT_PATH =', OUT_PATH.resolve())

PDF_DIR = /Users/igangsan/Desktop/K-IFRS
OUT_PATH = /Users/igangsan/Desktop/KIFRS_results/kifrs_combined_2.json


In [2]:
# 의존 패키지: pdfminer.six  (미설치시: pip install pdfminer.six)
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage

# 제목/번호(본문 또는 파일명), 문단번호 정규식
RE_TITLE_LINE = re.compile(r"(기업회계기준서\s*제\s*(\d{4})\s*호)\s*([^\n\r]*)", re.I)
RE_TITLE_ALT  = re.compile(r"K-IFRS[_\s-]*제?\s*(\d{4})\s*호[_\s-]*([^\n\r]*)", re.I)

# 문단번호: 괄호 없는 숫자 계열만 허용 (예: 1, 1., 1.1, 1.1., 한2, 한2.1, 한2.1.)
# 줄 시작 허용(들여쓰기 보정), 괄호 ((1) ...)는 문단번호로 취급하지 않음
RE_PARA_NUM   = re.compile(r"^\s*(?P<num>(?:한)?\d+(?:\.\d+)*)(?:\.)?\s+", re.M)

def clean_text(s: str) -> str:
    # 제어문자/여분 공백 정리 (줄바꿈은 '후처리'에서 쓰므로 여기선 보존)
    s = re.sub(r"[\u200b\ufeff\x00-\x08\x0b\x0c\x0e-\x1f]", " ", s)
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\s*\n\s*", "\n", s)  # 개행은 정규화만 하고 유지
    return s.strip()

def get_page_count(pdf_path: Path) -> int:
    with open(pdf_path, 'rb') as f:
        return sum(1 for _ in PDFPage.get_pages(f))

def extract_title_and_no_from_text(sample_text: str, fallback_name: str):
    std_no, title = None, None
    m1 = RE_TITLE_LINE.search(sample_text)
    if m1:
        std_no = m1.group(2)
        title  = (m1.group(3) or '').strip() or None
    if not std_no:
        m2 = RE_TITLE_ALT.search(fallback_name)
        if m2:
            std_no = m2.group(1)
            title  = (m2.group(2) or '').replace('_', ' ').strip() or None
    return std_no, title

In [3]:
import re

def postprocess_text(text: str, *, has_next_para: bool) -> str:
    """
    규칙:
    1) (중복 줄 삭제) 줄바꿈 제거 전에:
       - [규칙 A] '내용.' + 줄바꿈 + '내용' + (다음 문단번호) 형태면 '두 번째 내용' 삭제
       - [규칙 B] '(숫자) 내용' + 줄바꿈 + '내용' + (다음 문단번호) 형태면 '두 번째 내용' 유지
         (즉, 앞줄이 '(숫자) '로 시작하면 A를 적용하지 않음)
    2) 본문 안의 '- 숫자 -' 패턴 제거 (헤더/푸터 제거 용)
    3) 줄바꿈 문자 제거 시 앞뒤를 붙여서 한 줄로 만들기
    """

    # 0) 라인 단위 정리
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]

    # 1) 끝부분 중복 줄 삭제 (다음 문단번호가 실제로 존재할 때만)
    if has_next_para and len(lines) >= 2:
        a = lines[-2]  # 줄바꿈 '앞' 줄
        b = lines[-1]  # 줄바꿈 '뒤' 줄
        # 앞줄이 마침표로 끝나고, '(숫자) '로 시작하지 않으면 비교
        if a.endswith('.') and not re.match(r'^\(\d+\)\s+', a):
            a_base = re.sub(r'\.\s*$', '', a).strip()  # 마침표만 제거한 a
            if b == a_base:
                lines = lines[:-1]  # b 삭제

    s = "\n".join(lines)

    # 2) '- 숫자 -' 패턴 제거 (예: '- 12 -', '-12-', '- 3 -')
    s = re.sub(r'\s*-\s*\d+\s*-\s*', ' ', s)

    # 3) 줄바꿈 제거(붙여쓰기) + 다중 공백 정리
    s = s.replace('\n', '')
    s = re.sub(r'\s{2,}', ' ', s).strip()

    return s

In [4]:
def extract_pages(pdf_path: Path) -> Tuple[str, List[Tuple[int, int, int]]]:
    """
    각 페이지 텍스트를 추출해 하나의 큰 문자열로 합치고,
    페이지별 (start_offset, end_offset, page_no) 리스트를 반환.
    """
    texts = []
    ranges = []
    offset = 0
    page_count = get_page_count(pdf_path)
    for p in range(page_count):
        try:
            t = extract_text(str(pdf_path), page_numbers=[p]) or ''
        except Exception:
            t = ''
        t = clean_text(t)
        start = offset
        texts.append(t)
        offset += len(t) + 1   # 페이지 사이에 '\n' 하나 넣음
        end = offset
        ranges.append((start, end, p + 1))  # 1-based page
    full = "\n".join(texts)
    return full, ranges

def page_of_pos(pos: int, page_ranges: List[Tuple[int, int, int]]) -> int:
    for (s, e, page) in page_ranges:
        if s <= pos < e:
            return page
    return page_ranges[-1][2] if page_ranges else 1

def parse_pdf_into_paragraphs(pdf_path: Path) -> Dict[str, Any]:
    # 제목/번호: 1~2쪽 샘플 + 파일명 백업
    try:
        sample = (extract_text(str(pdf_path), page_numbers=[0]) or '') + \
                 (extract_text(str(pdf_path), page_numbers=[1]) or '')
    except Exception:
        sample = ''
    std_no, title = extract_title_and_no_from_text(sample, pdf_path.name)

    # 전체 텍스트 + 페이지 오프셋 맵
    full, pranges = extract_pages(pdf_path)
    if not full:
        return {
            'standard_no': std_no,
            'title': title,
            'source_file': pdf_path.name,
            'paragraphs': []
        }

    # 문단 경계: 문단번호 토큰 매칭 위치 ~ 다음 토큰 직전까지
    paragraphs = []
    matches = list(RE_PARA_NUM.finditer(full))
    for i, m in enumerate(matches):
        para_id = m.group('num')  # 괄호 없는 숫자 계열만
        start_text = m.end()
        end_text = matches[i+1].start() if i + 1 < len(matches) else len(full)
        raw_para = clean_text(full[start_text:end_text])
        if not raw_para:
            continue

        page = page_of_pos(m.start(), pranges)

        # 다음 문단번호가 실제로 존재하는지 플래그 (중복줄 삭제 규칙 활성화 여부)
        has_next = (i + 1 < len(matches))
        para_text = postprocess_text(raw_para, has_next_para=has_next)

        paragraphs.append({
            'para_id': para_id,
            'page': page,
            'text': para_text
        })

    return {
        'standard_no': std_no,
        'title': title,
        'source_file': pdf_path.name,
        'paragraphs': paragraphs
    }

In [5]:
def run_to_single_json(pdf_dir: Path = PDF_DIR, out_path: Path = OUT_PATH):
    """
    폴더 내 모든 PDF를 파싱해 하나의 JSON 파일로 저장.
    구조:
    {
      "documents": [
        {
          "standard_no": "1116",
          "title": "리스",
          "source_file": "K-IFRS_제1116호_리스.pdf",
          "paragraphs": [
            {"para_id":"1","page":5,"text":"..."},
            ...
          ]
        },
        ...
      ]
    }
    """
    pdfs = sorted([p for p in pdf_dir.glob('*.pdf')], key=lambda p: p.name)
    combined = {'documents': []}
    for i, pdf in enumerate(pdfs, 1):
        try:
            doc = parse_pdf_into_paragraphs(pdf)
            combined['documents'].append(doc)
            print(f"[{i}/{len(pdfs)}] parsed: {pdf.name} (paras: {len(doc['paragraphs'])})")
        except Exception as e:
            print(f"[ERROR] {pdf.name}: {e}")

    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(combined, f, ensure_ascii=False, indent=2)
    print("Saved ->", out_path)

run_to_single_json()

[1/28] parsed: K-IFRS_제1001호_재무제표_표시.pdf (paras: 145)
[2/28] parsed: K-IFRS_제1002호_재고자산.pdf (paras: 43)
[3/28] parsed: K-IFRS_제1007호_현금흐름표.pdf (paras: 62)
[4/28] parsed: K-IFRS_제1008호_회계정책_회계추정치_변경과_오류.pdf (paras: 54)
[5/28] parsed: K-IFRS_제1012호_법인세.pdf (paras: 108)
[6/28] parsed: K-IFRS_제1016호_유형자산.pdf (paras: 74)
[7/28] parsed: K-IFRS_제1019호_종업원급여.pdf (paras: 177)
[8/28] parsed: K-IFRS_제1020호_정부보조금의_회계처리와_정부지원의_공시.pdf (paras: 40)
[9/28] parsed: K-IFRS_제1021호_환율변동효과.pdf (paras: 57)
[10/28] parsed: K-IFRS_제1023호_차입원가.pdf (paras: 28)
[11/28] parsed: K-IFRS_제1024호_특수관계자공시.pdf (paras: 28)
[12/28] parsed: K-IFRS_제1026호_퇴직급여제도에_의한_회계처리와_보고.pdf (paras: 36)
[13/28] parsed: K-IFRS_제1028호_관계기업과_공동기업에_대한_투자.pdf (paras: 47)
[14/28] parsed: K-IFRS_제1032호_금융상품_표시.pdf (paras: 51)
[15/28] parsed: K-IFRS_제1033호_주당ᄋ